In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
data = pd.read_csv('Food.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        217 non-null    int64 
 1   Food      217 non-null    object
 2   Grams     217 non-null    object
 3   Calories  217 non-null    object
 4   Protein   217 non-null    object
 5   Category  217 non-null    object
dtypes: int64(1), object(5)
memory usage: 10.3+ KB


In [4]:
data['Grams'] = data['Grams'].str.replace(',', '.').astype(float)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        217 non-null    int64  
 1   Food      217 non-null    object 
 2   Grams     217 non-null    float64
 3   Calories  217 non-null    object 
 4   Protein   217 non-null    object 
 5   Category  217 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 10.3+ KB


In [6]:
data['Protein'] = data['Protein'].str.replace('t', '0').astype(float)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        217 non-null    int64  
 1   Food      217 non-null    object 
 2   Grams     217 non-null    float64
 3   Calories  217 non-null    object 
 4   Protein   217 non-null    float64
 5   Category  217 non-null    object 
dtypes: float64(2), int64(1), object(3)
memory usage: 10.3+ KB


In [10]:
data.head()

,ID,Food,Grams,Calories,Protein,Category
0,0,Cows' milk,976.000,660,32.0,Dairy products
1,1,Buttermilk,246.000,127,9.0,Dairy products
2,2,"Evaporated, undiluted",252.000,345,16.0,Dairy products
3,3,Fortified milk,1.419,"1,373",89.0,Dairy products
4,4,Powdered milk,103.000,515,27.0,Dairy products


### Rekomendasi


In [11]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
# Menghapus koma (,) dari kolom 'Calories' sebelum mengonversi ke integer
data['Calories'] = data['Calories'].str.replace(',', '').astype(int)

In [17]:
# Misalnya, df adalah DataFrame Anda
data['Grams'] = data['Grams'].astype(int)
data['Calories'] = data['Calories'].astype(int)
data['Protein'] = data['Protein'].astype(int)

In [22]:
# Menyimpan DataFrame yang sudah diubah ke file CSV
data.to_csv('data_food.csv', index=False)

In [18]:
#convert categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_features=encoder.fit_transform(data[['Category']])

In [19]:
#combine features
feature = data[['Calories']]
features = pd.concat([feature, pd.DataFrame(encoded_features)],axis=1)

In [20]:
#hitung cosine similarity
cosine_sim= cosine_similarity(features,features)

In [21]:
#fungsi to get top N food recommendation based od give feature
def model_food(query_Categori,query_Calories,top_n=10):
    query_encoded = encoder.transform([[query_Categori]])
    query_feature = [query_Calories] + list(query_encoded[0])

    similarity_scores = cosine_similarity([query_feature], features)
    similarity_food_indices = similarity_scores[0].argsort()[-top_n:][::1]
    similarity_food_info =data.iloc[similarity_food_indices]
    return similarity_food_info

# query_Protein = 20
query_Categori = 'Fish, Seafood'
query_Calories = 200
recommendations = model_food(query_Categori, query_Calories)
print('rekomendasi makanan :')
print(recommendations[['Food','Grams','Calories','Protein','Category']])

rekomendasi makanan :
                 Food  Grams  Calories  Protein       Category
82               Shad     85       170       20  Fish, Seafood
68                Cod    100       170       28  Fish, Seafood
85               Tuna     85       170       25  Fish, Seafood
77            Oysters    230       231      232  Fish, Seafood
80           Sardines     85       180       22  Fish, Seafood
84          Swordfish    100       180       27  Fish, Seafood
73            Halibut    100       182       26  Fish, Seafood
74            Herring    100       211       22  Fish, Seafood
71           Flounder    100       200       30  Fish, Seafood
70  Fish sticks fried    112       200       19  Fish, Seafood


C:\Users\bahbah\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [41]:
# Save the encoder and recommendation function to a pickled file
with open('recommendation_model.pkl', 'wb') as f:
    pickle.dump((encoder, model_food), f)